In [2]:
import cv2
import mediapipe as mp
import time

import math

import numpy as np
from mxnet import nd

# Mediapipe를 이용하여 FPS, Velocity, Angle을 realtime video에 나타내기

In [3]:
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

def dotproduct(v1, v2):
  d = nd.dot(v1, v2)
  return d.asscalar()

def length(v):
  return math.sqrt(dotproduct(v, v))


def angle(v1, v2):
  if (length(v1) * length(v2)) != 0 and -1 <= dotproduct(v1, v2) / (length(v1) * length(v2)) and  dotproduct(v1, v2) / (length(v1) * length(v2))<= 1:
      a = math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))
  else:
      a = 0
  return np.degrees(a)


#capture = cv2.VideoCapture("D:/simple_Pose/videos/golf_swing4.mp4")
#capture = cv2.VideoCapture("D:/simple_Pose/videos/test.mp4")

capture = cv2.VideoCapture(0,cv2.CAP_MSMF)
capture.set(cv2.CAP_PROP_FPS,30)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)



# used to record the time when we processed last frame
prev_frame_time = 0
 
# used to record the time at which we processed current frame
new_frame_time = 0


prev_right_wrist_x = 0
prev_right_wrist_y = 0


right_wrist_deg = 0
right_shoulder_deg = 0
right_elbow_deg = 0
right_thumb_deg = 0
right_index_deg = 0
right_pinky_deg = 0

count_check = 0

while True:
    sucess, img = capture.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = pose.process(imgRGB)
    #print(result.pose_world_landmarks)
    #mpDraw.draw_landmarks(img,result.pose_world_landmarks,mpPose.POSE_CONNECTIONS)
    
    if result.pose_landmarks:
        mpDraw.draw_landmarks(img, result.pose_landmarks,mpPose.POSE_CONNECTIONS)
        for id, lm in enumerate((result.pose_landmarks.landmark)):
            h,w,c = img.shape
            print(id,lm)
            cx,cy=int(lm.x*w), int(lm.y*h)
           # cv2.circle(img,(cx,cy),10,(255,0,0),cv2.FILLED)
            if id == 12:
                 rigrh_shoulder_x = lm.x 
                 rigrh_shoulder_y = lm.y
            elif id == 24:
                rigrh_hip_x = lm.x
                rigrh_hip_y = lm.y
                count_check = 1
            elif id == 14:
                right_elbow_x = lm.x
                right_elbow_y = lm.y
            elif id == 16:
                right_wrist_x = lm.x
                right_wrist_y = lm.y
            elif id == 22:          # 엄지
                right_thumb_x = lm.x
                right_thumb_y = lm.y
            elif id == 20:          # 검지     
                right_index_x = lm.x
                right_index_y = lm.y    
            elif id == 18:          # 약지
                right_pinky_x = lm.x
                right_pinky_y = lm.y
            if count_check == 1:
                right_wrist_deg = (np.abs(math.atan2(right_thumb_y-right_wrist_y, right_thumb_x-right_wrist_x) - math.atan2(right_wrist_y-right_elbow_y, right_wrist_x - right_elbow_x))*180)/math.pi
                right_shoulder_deg = (np.abs(math.atan2(right_hip_y-right_shoulder_y, right_hip_x-right_shoulder_x) - math.atan2(right_elbow_y-rigrh_shoulder_y, right_elbow_x - rigrh_shoulder_x))*180)/math.pi
                right_elbow_deg = (np.abs(math.atan2(right_shoulder_y-right_elbow_y, right_shoulder_x-right_elbow_x) - math.atan2(right_wrist_y-right_elbow_y, right_wrist_x-right_elbow_x))*180)/math.pi
                right_thumb_deg = (np.abs(math.atan2(right_thumb_y-right_wrist_y, right_thumb_x-right_wrist_x) - math.atan2(right_thumb_y-right_index_y, right_thumb_x-right_index_x))*180)/math.pi
                right_index_deg = (np.abs(math.atan2(right_index_y-right_wrist_y, right_index_x-right_wrist_x) - math.atan2(right_index_y-right_pinky_y, right_index_x-right_pinky_x))*180)/math.pi
                right_pinky_deg = (np.abs(math.atan2(right_pinky_y-right_wrist_y, right_pinky_x-right_wrist_x) - math.atan2(right_pinky_y-right_index_y, right_pinky_x-right_index_x))*180)/math.pi
                count_check = 0       

            
    prev_frame_coor = np.array([prev_right_wrist_x, prev_right_wrist_y])
    new_frame_coor = np.array([right_wrist_x, right_wrist_y])
    
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    fps1 = round(1/fps, 2)  # 한 프레임 당 초
    v = np.sqrt(np.sum(np.square(prev_frame_coor - new_frame_coor)))/fps1
    
    
    prev_frame_time = new_frame_time
    prev_right_wrist_x = right_wrist_x
    prev_right_wrist_y = right_wrist_y
    
    cv2.putText(img, "fps:" + str(int(fps)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)shoulder degree:" + str(int(right_shoulder_deg)), (70,100), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)elbow degree:" + str(int(right_elbow_deg)), (70,150), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)    
    cv2.putText(img, "(r)wrist degree:" + str(int(right_wrist_deg)), (70,200), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)thumb degree:" + str(int(right_thumb_deg)), (70,250), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)index degree:" + str(int(right_index_deg)), (70,300), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)pinky degree:" + str(int(right_pinky_deg)), (70,350), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    cv2.putText(img, "(r)wrist velocity:" + str(int(v)), (70,400), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0),3)
    
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
capture.release()
cv2.destroyAllWindows()

0 x: 0.5839320421218872
y: 0.7051421403884888
z: -1.5860393047332764
visibility: 1.0

1 x: 0.6037275791168213
y: 0.6260743141174316
z: -1.5915039777755737
visibility: 1.0

2 x: 0.620098352432251
y: 0.6239513754844666
z: -1.5917658805847168
visibility: 0.9999861717224121

3 x: 0.6365305781364441
y: 0.621544361114502
z: -1.5919872522354126
visibility: 1.0

4 x: 0.55888432264328
y: 0.623266875743866
z: -1.5954663753509521
visibility: 1.0

5 x: 0.5414102673530579
y: 0.6183883547782898
z: -1.5955348014831543
visibility: 0.9999933242797852

6 x: 0.5239267349243164
y: 0.6141615509986877
z: -1.596044898033142
visibility: 1.0

7 x: 0.6560609340667725
y: 0.6141871213912964
z: -1.337397575378418
visibility: 0.9999973773956299

8 x: 0.49549737572669983
y: 0.603827953338623
z: -1.355383276939392
visibility: 0.9999972581863403

9 x: 0.607660710811615
y: 0.7569923996925354
z: -1.4647459983825684
visibility: 0.9999586343765259

10 x: 0.5543982982635498
y: 0.7530871033668518
z: -1.4692782163619995
visi

NameError: name 'right_hip_y' is not defined